## Relational Embedding Example Functionality on MIT DWH

In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
from relational_embedder.data_prep import data_prep_utils as dpu
import numpy as np
from relational_embedder import api
from relational_embedder.api import Fabric
from relational_embedder.api import SIMF
import word2vec
import pickle
import itertools
import pandas as pd
path_to_row_we_model = '/data/raulcf/relemb/mitdwh/mitdwh_model/row_we_db2vec_75_50_n20_i40.bin'
row_we_model = word2vec.load(path_to_row_we_model)
path_to_col_we_model = '/data/raulcf/relemb/mitdwh/mitdwh_model/col_we_w2v_75_50_i40.bin'
col_we_model = word2vec.load(path_to_col_we_model)
path_to_row_relemb = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/row.pkl'
with open(path_to_row_relemb, "rb") as f:
    row_relemb = pickle.load(f)
path_to_col_relemb = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/col.pkl'
with open(path_to_col_relemb, "rb") as f:
    col_relemb = pickle.load(f)
path_to_row_hubness = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/row_hubness.pkl'
with open(path_to_row_hubness, "rb") as f:
    word_row_hubness = pickle.load(f)
path_to_col_hubness = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/col_hubness.pkl'
with open(path_to_col_hubness, "rb") as f:
    word_col_hubness = pickle.load(f)
path_to_relations = '/data/raulcf/mitdwhdata/'
fabric = Fabric(row_we_model, col_we_model, row_relemb, col_relemb, path_to_relations, word_hubness=word_row_hubness)

# Find related and similar content

#### Find related entities

In [103]:
kw = "madden_samuel_r"
res = fabric.topk_related_entities(kw, k=10)
res

[('madden@csail_mit_edu', 0.95556640625),
 ('srmadden', 0.935546875),
 ('http://db_csail_mit_edu/madden', 0.92138671875),
 ('6172586643', 0.90478515625),
 ('madden', 0.677734375),
 ('sbrinton', 0.67236328125),
 ('32-g938', 0.67041015625),
 ('sbrinton@mit_edu', 0.66943359375),
 ('920051802', 0.6640625),
 ('schendel@mit_edu', 0.6611328125)]

#### Find more entities like the input

In [4]:
kw = "madden_samuel_r"
res = fabric.more_entities_like(kw, k=10)
res

[('cadogan_karen_m', 0.9794921875),
 ('lynn_jennifer_j', 0.96435546875),
 ('allan_nicholas', 0.96435546875),
 ('regev_aviv', 0.9560546875),
 ('lapide_lawrence', 0.94482421875),
 ('glennerster_rachel', 0.94384765625),
 ('majewski_janusz_j', 0.93505859375),
 ('santos_ad\x8fle_naud_', 0.9345703125),
 ('santos_adèle_naudé', 0.91162109375),
 ('hadley_franklin_e_w', 0.89453125)]

#### Find relations relevant to the input entity

In [5]:
kw = "madden_samuel_r"
kwv = fabric.col_vector_for(kw)
res = fabric.top_relevant_relations(kwv, k=10)
res

[('short_moira_list_detail.csv', 0.479),
 ('Iap_subject_session.csv', 0.4482),
 ('short_library_reserve_catalog.csv', 0.4226),
 ('Person_auth_area.csv', 0.4065),
 ('Sdo_area_units.csv', 0.3906),
 ('Ir_institution.csv', 0.3838),
 ('Sdo_datums_old_snapshot.csv', 0.379),
 ('Sdo_datums_old_format.csv', 0.375),
 ('Sdo_dist_units.csv', 0.357),
 ('Sdo_coord_op_methods.csv', 0.351)]

#### Find columns relevant to the input entity

In [6]:
kw = "madden_samuel_r"
kwv = fabric.col_vector_for(kw)
res = fabric.topk_relevant_columns(kwv, k=10)
res

[('Responsible Faculty Name', 'short_tip_subject_offered.csv', 0.652),
 ('Responsible Faculty Name',
  'short_course_catalog_subject_offered.csv',
  0.641),
 ('Responsible Faculty Name', 'short_subject_offered_summary.csv', 0.6255),
 ('Moira List Member Full Name', 'short_moira_list_detail.csv', 0.6157),
 ('Responsible Faculty Name', 'short_subjects_offered.csv', 0.615),
 ('Hr Org Level3 Id', 'Hr_org_unit_new.csv', 0.6055),
 ('Hr Org Level3 Id', 'Hr_org_unit.csv', 0.6055),
 ('Responsible Faculty Name', 'Library_subject_offered.csv', 0.603),
 ('Icon Gif Name', 'Cis_hass_attribute.csv', 0.6016),
 ('Coord Op Name', 'Sdo_coord_ops.csv', 0.59)]

#### Find rows relevant to the input

In [7]:
kw = "madden_samuel_r"
kwv = fabric.row_vector_for(kw)
res = fabric.topk_relevant_rows(kwv, k=10)
res

[(User Name                       SRMADDEN
  Has Financial Auth                     Y
  Has Hr Full Auth                       N
  Has Hr Limited Auth                    N
  Has Payroll Auth                       N
  Warehouse Load Date    01/08/16 12:00 AM
  Name: 25079, dtype: object, 'Person_auth_area.csv', 0.935436790859967),
 (Mit Id                                                                987515930
  Last Name                                                                Madden
  First Name                                                               Samuel
  Middle Name                                                                   R
  Full Name                                                      Madden, Samuel R
  Directory Full Name                                            Madden, Samuel R
  Office Location                                                         32-G938
  Office Phone                                                         6172586643
  Directory 

#### Find rows relevant to the input - diversity output so it involves more heterogeneous relations

In [9]:
kw = "madden_samuel_r" 
kwv = fabric.row_vector_for(kw)
res = fabric.topk_relevant_rows_diverse(kwv, k=10)
res

[(User Name                       SRMADDEN
  Has Financial Auth                     Y
  Has Hr Full Auth                       N
  Has Hr Limited Auth                    N
  Has Payroll Auth                       N
  Warehouse Load Date    01/08/16 12:00 AM
  Name: 25079, dtype: object, 'Person_auth_area.csv', 0.935436790859967),
 (Mit Id                                                                987515930
  Last Name                                                                Madden
  First Name                                                               Samuel
  Middle Name                                                                   R
  Full Name                                                      Madden, Samuel R
  Directory Full Name                                            Madden, Samuel R
  Office Location                                                         32-G938
  Office Phone                                                         6172586643
  Directory 

#### Find rows relevant to input within table  # does not seem that useful

In [87]:
kw = "madden_samuel_r"
kwv = fabric.row_vector_for(kw)
relation = 'Buildings.csv'
rows_summary = fabric.topk_related_rows_in_relation(kwv, relation, k=5)
rows_summary

[(Building Key                                                     OC22
  Building Number                                                  OC22
  Building Name                     Westford Optics Facility (Haystack)
  Building Street Address                                 50  GROTON RD
  Building Mailing Address                                          NaN
  Bldg Gross Square Footage                                      881.78
  Bldg Assignable Square Footage                                 794.16
  Building Counter                                                    1
  Warehouse Load Date                                 01/08/16 05:21 AM
  Name: 128, dtype: object, 'Buildings.csv', 0.3062),
 (Building Key                                    W57
  Building Number                                 W57
  Building Name                          Building W57
  Building Street Address               169 VASSAR ST
  Building Mailing Address                        NaN
  Bldg Gross Square Footage 

# Summarizing relations

#### Summarizing a relation in terms of its rows

In [89]:
relation = "Sis_department.csv"
rows = fabric.relation_in_rows_summary(relation, k=3)
rows

[Department Code                                                     4
 Department Name                                          Architecture
 Department Full Name                                  4 -Architecture
 School Code                                                         A
 School Name                                 Architecture and Planning
 Dept Budget Code                                                31000
 Is Degree Granting                                                  Y
 Dept Name In Commencement Bk               Department of Architecture
 School Name In Commencement Bk    School of Architecture and Planning
 Department Name History                                           NaN
 Department Last Activity Date                       02/17/94 12:00 AM
 Dlc Key                                                        D_ARCH
 Warehouse Load Date                                 01/08/16 12:10 AM
 Name: 0, dtype: object,
 Department Code                                    

#### Summarizing a database in terms of its relations

In [81]:
relation_sample = fabric.db_in_relations_summary(k=10)
relation_sample

['Time_day.csv',
 'Fclt_rooms.csv',
 'Sdo_coord_ops.csv',
 'Zip_usa.csv',
 'Sdo_available_elem_ops.csv',
 'Library_reserve_matrl_detail.csv',
 'Buildings.csv',
 'Sdo_datums_old_format.csv',
 'Master_dept_dcode_parent.csv',
 'Sdo_non_available_elem_units.csv']

# Explain relationship between...

#### Explain the relationship between two tables by showing me entities that relate them

In [104]:
relation1 = "Employee_directory.csv"
relation2 = 'Fclt_organization.csv'
res = fabric.entity_evidence_related_tables(relation1, relation2, k=10)
res

['shah_darshita_(dipa)_n',
 'teaching_&_learning_laboratory',
 'http://tll_mit_edu',
 '441700_0',
 'leann',
 'dobranski',
 'teaching_and_learning_laboratory',
 'darshita_(dipa)_n',
 'integrated_studies_program',
 '060085']

#### Explain the relationship between two tables by showing me pairs of columns related to each other

In [105]:
relation1 = "Employee_directory.csv"
relation2 = 'Fclt_organization.csv'
res = fabric.column_evidence_related_tables(relation1, relation2, k=10)
res

[(('Department Number', 'Organization Number'), 0.5947),
 (('Department Number', 'Hr Department Code Old'), 0.5947),
 (('Department Number', 'Major Org'), 0.593),
 (('Department Number', 'Organization Sort'), 0.4534),
 (('Mit Id', 'Organization Sort'), 0.4407),
 (('Department Number', 'Hr Org Unit Id'), 0.4307),
 (('Mit Id', 'Major Org'), 0.4075),
 (('Mit Id', 'Organization Number'), 0.4062),
 (('Mit Id', 'Hr Department Code Old'), 0.4053),
 (('Mit Id', 'Hr Org Unit Id'), 0.4045)]

#### Explain the relationship between two tables through pairs of rows are related to each other

In [106]:
relation1 = "Employee_directory.csv"
relation2 = 'Fclt_organization.csv'
res = fabric.row_evidence_related_tables(relation1, relation2, k=10)
res

[(Mit Id                                                            924102952
  Last Name                                                         Dobranski
  First Name                                                            Leann
  Middle Name                                                             NaN
  Full Name                                                  Dobranski, Leann
  Directory Full Name                                        Dobranski, Leann
  Office Location                                                     E39-207
  Office Phone                                                     6172533371
  Directory Title               Assistant Director, Teaching and Learning Lab
  Primary Title                 Assistant Director, Teaching and Learning Lab
  Department Number                                                    441700
  Department Name                            Teaching and Learning Laboratory
  Krb Name                                                      

#### Explain the relationship between two columns in terms of entities that relate them

In [107]:
col1 = fabric.RE_C['Employee_directory.csv']['columns']['Department Name']
col2 = fabric.RE_C['Fclt_organization.csv']['columns']['Hr Department Name']
res = fabric.entity_evidence_related_columns(col1, col2)
res

[('professional_education_international_programs', 0.5380859375),
 ('school_development_services_office_of', 0.5791015625),
 ('urban_studies_&_planning_department_of', 0.5673828125),
 ('aeronautics_and_astronautics_department_of', 0.552734375),
 ('electrical_engineering_&_computer_science_dept_of', 0.54736328125),
 ('archaeology_department_of', 0.541015625),
 ('biomedical_innovation_the_center_for', 0.56005859375),
 ('facilities_utilities', 0.56494140625),
 ('civil_and_environmental_engineering_department_of', 0.54931640625),
 ('mechanical_engineering_department_of', 0.55615234375),
 ('biological_engineering_department_of', 0.55078125),
 ('linguistics_and_philosophy_department_of', 0.5546875),
 ('political_science_department_of', 0.55859375),
 ('chemistry_department_of', 0.54443359375),
 ('brain_and_cognitive_sciences_department_of', 0.55859375),
 ('student_assistance_services', 0.55126953125),
 ('student_financial_records', 0.546875),
 ('system_generated', 0.544921875),
 ('undergradua